In [2]:
import maup
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

In [3]:
cook_cnty_blocks = gpd.GeoDataFrame.from_file("../data/tl_2010_17031_tabblock10/tl_2010_17031_tabblock10.shp")
blk_data = pd.read_csv("../data/10decPL_chicago_blocks_summary.csv")
blk_geoids = list(map(lambda geoid: geoid.split("US")[1], blk_data["GEO_ID"]))
blk_data["GEO_ID"] = blk_geoids

In [31]:
blk_gdf = cook_cnty_blocks[cook_cnty_blocks["GEOID10"].isin(blk_data["GEO_ID"])].to_crs("EPSG:3435")
prct_gdf = gpd.GeoDataFrame.from_file("../data/precincts_shapefile/PRECINCTS_2012.shp")

In [5]:
clipped_blks = gpd.clip(blk_gdf, prct_gdf)

/Users/hazelnb/opt/anaconda3/lib/python3.9/site-packages/geopandas/tools/clip.py:66: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[


In [43]:
clipped_blks['prcnt_assignment'] = maup.assign(clipped_blks, prct_gdf)

/Users/hazelnb/opt/anaconda3/lib/python3.9/site-packages/maup/indexed_geometries.py:17: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  self.spatial_index = STRtree(self.geometries)
/Users/hazelnb/opt/anaconda3/lib/python3.9/site-packages/maup/indexed_geometries.py:17: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  self.spatial_index = STRtree(self.geometries)


In [44]:
combined_blk_data = pd.merge(clipped_blks, blk_data, how="inner", left_on="GEOID10", right_on="GEO_ID")
# combined_blk_data.to_file("test3.shp")

In [52]:
vap_cols = list(filter(lambda col: "_vap" in col, combined_blk_data.columns))
len(list(combined_blk_data.groupby(combined_blk_data['precinct']))[4][1]["GEOID10"])

14

In [46]:
prct_gdf[vap_cols] = combined_blk_data[vap_cols].groupby(combined_blk_data['prcnt_assignment']).sum()

In [56]:
print(max(prct_gdf["tot_vap"]))

11385.0
